# Notebook para ejecutar el proceso de extracción de datos del sítio https://www.espaciourbano.com/

Librerías

In [2]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

Lectura de archivos en utilidades

In [5]:
# Carga de listado de users agents
user_agets = pd.read_csv("utils/user-agent.csv")

# Carga de listado de proxies
with open("utils/proxies.json") as json_file:
    proxies = json.load(json_file)
proxies = proxies.get("proxies")

**Definiciones de URLs**

El sitio espacio urbano contiene 5 zonas para casas en alquiler en la ciudad de Medellín:
- Centro
- Poblado
- Belen
- Laureles
- San Antonio de Prado

Cada una de las zonas contiene una URL producto de interactuar con el sitio y hacer los respectivos filtros por zonas, además cada una de estas puede tener resultados en n páginas, estas urls se definen a continuación

In [6]:
# URL Principal
head_url = "https://www.espaciourbano.com/"

# URL Zona Centro
urls_zona_centro   = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10000&pTipoInmueble=1&nCiudad=Medellin%20Zona%201%20-%20Centro",          # Página 1
                      "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10000&pTipoInmueble=1&nCiudad=Medellin+Zona+1+%2D+Centro&offset={limit}"] # Página n

# URL Zona Poblado
urls_poblado = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10027&pTipoInmueble=1&nCiudad=Medellin%20Zona%202%20-%20El%20Poblado",         # Página 1
                "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10027&pTipoInmueble=1&nCiudad=Medellin+Zona+2+%2D+El+Poblado&offset={limit}"]  # Página n

# URL Zona Belen
urls_belen = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10029&pTipoInmueble=1&nCiudad=Medellin%20Zona%204%20-%20Belen",          # Página 1
              "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10029&pTipoInmueble=1&nCiudad=Medellin+Zona+4+%2D+Belen&offset={limit}"] # Página n

# URL Laureles
urls_laureles = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10028&pTipoInmueble=1&nCiudad=Medellin%20Zona%203%20-%20Laureles",          # Página 1
                 "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10028&pTipoInmueble=1&nCiudad=Medellin+Zona+3+%2D+Laureles&offset={limit}"] # Página n

# URL San Antonio de Prado
urls_san_antonio = ["https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10041&pTipoInmueble=1&nCiudad=San%20Antonio%20de%20Prado",          # Página 1
                    "https://www.espaciourbano.com/resumen_ciudad_arriendos.asp?pCiudad=10041&pTipoInmueble=1&nCiudad=San+Antonio+de+Prado&offset={limit}"] # Página n

**Funciones:**

La información de cada uno de los inmuebles esta dividida en
- Precio
- Zona
- Comonidades
- Caracteristicas

Se define una función que realiza la petición a cada sitio y extrae la información de cada inmueble con el respectivo código de registro de la página

In [9]:
def get_properties_info(url: str) -> tuple:
    """
    Función para realizar la petición a una url especifica
    y devolver los datos de cada inmueble

    PARAMS
        url: str
            url de una página con el listado de los inmuenles en arriendo
    RETURN
        (cod, lease) tuple
            código de un inmueble con su respectivo grupos de valores
            precio, zona, comodidades y caracteristicas
    """
    # Variar el user agent por petición
    headers = {'User-Agent': user_agets.sample(1)["user-agent"].values[0]}

    # Consulta y carga de la url de detalle
    detail = requests.get(url.replace("Ficha","ficha"), headers=headers)

    soup_detail =BeautifulSoup(detail.content,'lxml')

    # Obtención del código de vivienda, precio y zona
    price_zone = soup_detail.find_all('div', attrs={'class':'text-center'})
    price_zone = price_zone[1].find_all('h3')
    code = soup_detail.find_all('div', attrs={'class':'text-center'})[0].text.strip()
    price = price_zone[0].text.replace("\r\n","").strip()
    zone = price_zone[2].text

    # Obtención de las comodidades
    comforts = []
    comforts_div = soup_detail.find('div', attrs={'class' : 'col-lg-4'}).find_all('p')
    for com in comforts_div:
        comfort = com.text.strip()
        if comfort != "": 
            comforts.append(comfort)

    
    # Obtención de las características
    characteristics = {}
    characteristics_div = soup_detail.find('div', attrs={'class' : 'col-lg-8'}).find('table').find_all('tr')
    for cha in characteristics_div:
        row_cha = cha.find_all('td')
        characteristics[row_cha[0].text.strip()] = row_cha[1].text.strip()

    lease = {
        "precio" : price,
        "zona" : zone,
        "comodidades" : comforts,
        "caracteristicas" : characteristics
    }

    # retorno de valores
    return code, lease

In [12]:
def get_zone_info(urls: list,
                  pages: int,
                  path: int,
                  zona: str = "") -> tuple:
    """
    Función para consultar la información de los inmuebles por cada zona
    PARAMS
        urls:lis
            Lista de las urls por zona
        pages: int
            Cantidad de páginas que tiene cada zona
        path: int
            Salto de página de cada sitio
        zona: str
            Nombre de zona que se esta procesando
    RETURN
        leases: dict
            Información de los inmuebles por zona
        characteristics: set
            Consolidado de los tipos de características por zona
        comforts: set
            Consolidado de las diferentes comodidades por zona
    """

    # Diccionario para almacenar los resultados
    leases = {}
    # Conjuntos para almacenar los consolidados de
    # Comodidades y características
    characteristics = set()
    comforts = set()

    print(f"Consultando Zona: {zona}")

    for i in range(0, pages):
        
        user_agets.sample(1)["user-agent"].values[0]
        # Se selecciona un agente de forma aleatoria
        headers = {'User-Agent': user_agets.sample(1)["user-agent"].values[0]}
        
        # Selección de la página a consultar
        if i == 0:
            url = urls[0]
        else:
            url = urls[1].format(pages = i*path)

        print(f"-- Consultando página : {i*path}")
        response = requests.get(url,headers=headers)
        
        if response.status_code == 200:
            # Por medio de BeautifulSoup se extrae el contendio de la pagina 
            soup_list = BeautifulSoup(response.content,'lxml')

            # Cargar todas las clases div con para obtener el detalle de las viviendas
            urls_details = soup_list.find_all('div', attrs={'class' : 'col-sm-4'})
            urls_details.pop(0)
            for url_detail in urls_details:
                a = url_detail.find('a')
                url = head_url + a.get('href')
                code, lease = get_properties_info(url)
                leases[code] = lease
                characteristics = characteristics.union(set(lease["caracteristicas"].keys()))
                comforts = comforts.union(set(lease["comodidades"]))
        else:
            print("ERROR: Falla consultando url: {}".format(url))
    return leases, characteristics, comforts

In [13]:
def write_file(data: set, file_name: str) -> None:
    """
    Función para imprimir datos en un archivo .txt
    PARAMS
        data: set
            Conjunto con los datos a imprimir en el archivo
        file_name: str
            Nombre del archivo a crear
    RETUN
    
    """
    # Transfomación de los datos a una lista
    data = list(data)
    # Crear archivo sobre el cual se va a escribir
    with open(f'{file_name}.txt', 'w') as f:
        # Ciclo para escribir cada línea en el archivo
        for line in data:
            f.write(line + '\n')
    f.close

***Ejecución de proceso de peticiones o requests:***

Teniendo las urls y las funciones se procede a ejecutar el proceso de extracción de información por cada zona, en este proceso se deben generar 3 archivos:
- .json: archivo json por zona con la informacion de los inmuebles
- .txt: archivo con el consolidado de las características de todos los inmuebles de cada zona
- .txt: archivo con el consolidado de las comodidades de todos los inmuebles de cada zona

Los archivos de comodidades y características se deben generar para crear una variable booleana con cada uno de los valores de estos archivos, por tanto por cada inmueble se tendrá una variable booleana que indique si tiene o no dicha caracteristica

Estos resultados serán almacenados en la carpeta _resultados_request_

**Scraping: Zona San Antonio de Prado**

In [14]:
# Parametros
urls = urls_san_antonio
pages = 2 # Número de páginas del sitio
path = 50 # Indica el salto de página
file_name = "resultados_request/zona_5_san_antonio"
characteristics_file = "resultados_request/zona_5_san_antonio_carac"
comforts_file = "resultados_request/zona_5_san_antonio_com"

In [ ]:
# Ejecución del scraping
leases, characteristics, comforts = get_zone_info(urls,
                                                  pages,
                                                  path,
                                                  zona = "San Antonio de Prado")

In [ ]:
with open(f"{file_name}.json", "w", encoding='utf-8') as outfile:
    json.dump(leases, outfile)

write_file(characteristics, characteristics_file)
write_file(comforts_file, comforts_file)